[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/wattwatchers/le_completeness_analysis/blob/main/le_completeness_analysis/analysis_colab.ipynb)

# Device id(s) configuration.

There are 2 options to configure devices for analysis.
1. All devices associated with the API key configured in the .env file
2. A list of device ids


In [255]:
# API key for the Public REST API
# If you want to analyse all devices associated with the API key, set the value of DEVICE_IDS below to an empty list ([])
API_KEY: str = ""

# If you want to analyse a subset of devices, enter the device ids inside the brackets ([]) like this:
# [ "DD1234567890", "DD2345678901"]

DEVICE_IDS: list = []

# Period configuration

Configure the time period to analyse.


In [256]:
TIMEZONE: str =   "Australia/Sydney"  # The timezone the period is defined in
START_DATE: str = "2025-02-20"        # Period starts at start of START_DATE (date string in the format <YYYY-MM-DD>)
END_DATE: str =   "2025-03-27"        # Period ends at the end of END_DATE (ate string in the format <YYYY-MM-DD>)
                                      # Has to be today or earlier. 
                                      # If today, analysis will be up to the most recent 5 minute boundary at the time the notebook is run.

# Threshold configuration

Configure the data completeness threshold (as a percentage) under which a device is considered problematic

In [257]:
DATA_COMPLETENESS_THRESHOLD: float = 80

# Other config (you probably won't need to touch these)

In [258]:
MAX_TPS: int                    = 10            # Maximum # of API requests per second
ENVIRONMENT: str                = "production"  # API environment (staging or production)
ANALYSE_UNAGGREGATED_DATA: bool = True          # When False, only data aggregated to daily granularity is generated and analysed
                                                # When True, data is also kept and analysed at 5m granularity and. Only do this for small sets of devices / short time periods
                                                # When 50 or more devices are analysed, unaggregated data will not be analysed, regardless of this configuration 

# API client

In [ ]:
import os
import logging
import re

import plotly 
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import pandas as pd
%pip install pendulum
import pendulum
%pip install itables
from itables import show, JavascriptFunction, JavascriptCode

In [260]:
def get_logger(logging_level: str = "INFO") -> logging.Logger:
    logger = logging.getLogger("notebook")
    logger.setLevel(logging_level)

    # loggers are cached, so if we call this from multiple places we end up with multiple handlers
    if logger.hasHandlers():
        return logger

    stdout_handler = logging.StreamHandler()
    formatter: logging.Formatter = logging.Formatter("%(asctime)s %(levelname)s %(message)s")
    stdout_handler.setFormatter(formatter)
    logger.addHandler(stdout_handler)
    return logger

In [261]:
import time
from functools import partialmethod
from typing import Any

import httpx
from pendulum import DateTime

JSONType = None | bool | int | float | str | list[Any] | dict[str, Any]


class RestError(Exception):
    """
    Error from REST API Client.
    """

    def __init__(
        self,
        message: str,
        request: httpx.Request,
        response: httpx.Response | None = None,
    ):
        super().__init__(message)
        self.request: httpx.Request = request
        self.response: httpx.Response | None = response


class RestAPIClient:

    def __init__(self, base_url: str, requests_per_sec_max: int, **session_kwargs):
        self._base_url = base_url
        self._client = httpx.Client()
        self._requests_per_sec_max = requests_per_sec_max
        self._last_request_time: DateTime | None = None

        for key, value in session_kwargs.items():
            setattr(self._client, key, value)

    def __enter__(self):
        return self

    def __exit__(self, *_):
        self.close()

    def close(self):
        return self._client.close()

    def _throttler(self):
        """
        This method throttles API request based on when the last request was made and the number of maximum number of requests per second configured.
        (the actual frequency of requests can be lower than the maximum allowed if requests take longer to complete than the minimum interval
        between requests)
        """
        if self._last_request_time is None:
            return
        time_since_last_request = self._last_request_time.diff().total_seconds()
        wait_duration = max(0, 1 / self._requests_per_sec_max - time_since_last_request)
        if wait_duration > 0:
            time.sleep(wait_duration)

    def request(self, method: str, path: str, **kwargs) -> tuple[JSONType, RestError]:
        self._throttler()
        try:
            resp = self._client.request(method, f"{self._base_url}/{path}", **kwargs)
            resp.raise_for_status()
            if len(resp.text) == 0:
                return None, None
            return (resp.json(), None)
        except httpx.HTTPStatusError as http_error:
            error_message = http_error.response.json().get("message", "")
            error = RestError(
                f"Error response {http_error.response.status_code} while requesting {http_error.request.url!r}: {error_message}",
                http_error.request,
                http_error.response,
            )
            return (None, error)
        except httpx.RequestError as err:
            error = RestError(
                f"An error occurred while requesting {err.request.url!r}.", err.request
            )
            return (None, error)
        finally:
            self._last_request_time = pendulum.now()

    get = partialmethod(request, "GET")
    post = partialmethod(request, "POST")
    put = partialmethod(request, "PUT")
    patch = partialmethod(request, "PATCH")
    delete = partialmethod(request, "DELETE")
    head = partialmethod(request, "HEAD")
    options = partialmethod(request, "OPTIONS")


In [262]:
from dataclasses import dataclass
from enum import Enum
import json

@dataclass
class TimeInterval:
    """
    Data class for a time interval
    """

    timestamp_start: int
    timestamp_end: int


class Granularity(Enum):
    """
    Enum for different LE granularities
    """

    FIVE_MINS = "5m"
    FIFTEEN_MINS = "15m"
    THIRTY_MINS = "30m"
    HOUR = "hour"
    DAY = "day"
    WEEK = "week"
    MONTH = "month"


class CallerError(Exception):
    """
    Error by caller of the client.
    """


class PublicApiClient(RestAPIClient):

    def __init__(
        self,
        environment: str,
        api_key: str,
        requests_per_sec_max: int,
        logger: logging.Logger,
    ):
        match environment:
            case "production" | "prod":
                base_url = "https://api-v3.wattwatchers.com.au"
            case "staging":
                base_url = "https://api-v3-stage.wattwatchers.com.au"
            case _:
                # fallback is prod
                base_url = "https://api-v3.wattwatchers.com.au"
        headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
        }
        # TODO: use a considered value for number of requests per second
        super().__init__(base_url, requests_per_sec_max, headers=headers)
        self._logger = logger

    def get_devices_list(self) -> tuple[list | None, RestError | None]:
        """
        Retrieves all device ids associated with the API key
        """
        result = super().get("devices")
        return result

    def get_device_status(self, device_id: str) -> tuple[dict | None, RestError | None]:
        """
        Retrieves the status of the device associated with the device_id
        """
        result = super().get(f"devices/{device_id}")
        return result

    def patch_device_status(
        self, device_id: str, payload: dict
    ) -> tuple[dict | None, RestError | None]:
        """
        Patches the device status of the device associated with the device_id
        Used (among other things) to update WiFi credentials
        """
        result = super().patch(f"devices/{device_id}", data=json.dumps(payload))
        return result

    def update_wifi_credentials(
        self, device_id: str, ssid: str | None = None, psk: str | None = None
    ) -> tuple[dict | None, RestError | None]:
        """
        Updates the WiFi credentials of the device associated with the device_id
        If successful, this will cause the device to switch to WiFi comms.
        """
        if ssid is None and psk is None:
            # No credential details provided, return error
            return (
                None,
                CallerError(
                    "Request to update WiFi credentials requires at least one of SSID and PSK to be defined."
                ),
            )

        payload = {"comms": {"wifi": {}}}
        if not (ssid is None):
            payload["comms"]["wifi"]["ssid"] = ssid
        if not (psk is None):
            payload["comms"]["wifi"]["psk"] = psk

        return self.patch_device_status(device_id, payload)

    def reset_wifi_credentials(
        self, device_id: str
    ) -> tuple[dict | None, RestError | None]:
        """
        Resets the WiFi credentials of the device associated with the device_id
        This will cause the device to switch to cellular comms.
        """
        return self.update_wifi_credentials(device_id, "", "")

    def change_switch_state(
        self, device_id: str, switch_id: str, target_state: str
    ) -> tuple[dict | None, RestError | None]:
        """
        Changes the switch state of the switch with id `switch_id` on the device with
        id `device_id` to state `target_state`.
        """
        payload = {
            "id": device_id,
            "switches": [{"id": switch_id, "state": target_state}],
        }
        return self.patch_device_status(device_id, payload)

    def update_se_reporting_interval(
        self, device_id: str, reporting_interval: int
    ) -> tuple[dict | None, RestError | None]:
        """
        Update the SE reporting interval for the device to the requested value
        """
        payload = {"shortEnergyReportingInterval": reporting_interval}
        return super().post(
            f"devices/{device_id}/reporting-interval", data=json.dumps(payload)
        )

    def get_latest_se(
        self, device_id: str, energy_unit: str | None = "kW"
    ) -> tuple[dict | None, RestError | None]:
        if energy_unit is not None and energy_unit in ["kW", "kWh"]:
            params = {"convert[energy]": energy_unit}
            return super().get(f"short-energy/{device_id}/latest", params=params)
        return super().get(f"short-energy/{device_id}/latest")

    def _max_interval_for_granularity(self, granularity: Granularity) -> int:
        """
        Returns the maximum interval for a single energy request based on the granularity
        """
        MAX_INTERVALS_DAYS = {
            Granularity.FIVE_MINS: 7,
            Granularity.FIFTEEN_MINS: 14,
            Granularity.THIRTY_MINS: 31,
            Granularity.HOUR: 90,
            Granularity.DAY: 3 * 365,  # ≈ 3 years
            Granularity.WEEK: 5 * 365,  # ≈ 5 years
            Granularity.MONTH: 10 * 365,  # ≈ 10 yers
        }
        return MAX_INTERVALS_DAYS.get(granularity, 7) * 24 * 3600

    def _calculate_intervals_for(
        self, granularity: Granularity, timestamp_start: int, timestamp_end: int
    ) -> list[tuple[int, int]]:
        """
        Batches an interval based on the maximum interval per request for the given granularity.
        """
        batch_interval = self._max_interval_for_granularity(granularity)
        intervals = [
            TimeInterval(batch_start, min(batch_start + batch_interval, timestamp_end))
            for batch_start in range(timestamp_start, timestamp_end, batch_interval)
        ]
        return intervals

    def _load_energy(
        self,
        endpoint: str,
        device_id: str,
        intervals: list[TimeInterval],
        unit: str = "kWh",
        granularity: Granularity | None = None,
    ) -> tuple[list | None, RestError | None]:

        energy_data = []
        for interval in intervals:
            params = {
                "fromTs": interval.timestamp_start,
                "toTs": interval.timestamp_end,
                "convert[energy]": unit,
            }
            if granularity is not None:
                params["granularity"] = granularity.value

            self._logger.info(
                f"load from {interval.timestamp_start} to {interval.timestamp_end} for {device_id}"
            )
            (result, error) = super().get(endpoint, params=params)
            if error is not None:
                self._logger.error(
                    f"Error retrieving LE data for {device_id} between {interval.timestamp_start} and {interval.timestamp_end}: {error}"
                )
                return (None, error)

            if result is not None:
                energy_data.extend(result)

        return (energy_data, None)

    def load_long_energy(
        self,
        device_id: str,
        timestamp_start: int,
        timestamp_end: int,
        granularity: Granularity = Granularity.FIVE_MINS,
        unit: str = "kWh",
    ) -> tuple[list | None, RestError | None]:

        intervals = self._calculate_intervals_for(
            granularity, timestamp_start, timestamp_end
        )
        return self._load_energy(
            f"long-energy/{device_id}", device_id, intervals, unit, granularity
        )

    def get_first_le(self, device_id: str) -> tuple[list | None, RestError | None]:
        result = super().get(f"long-energy/{device_id}/first")
        return result

    def load_short_energy(
        self,
        device_id: str,
        timestamp_start: int,
        timestamp_end: int,
        unit: str = "kWh",
    ) -> tuple[list | None, RestError | None]:

        max_interval = 12 * 3600  # maximum request interval for SE is 12 hours
        intervals = [
            TimeInterval(batch_start, min(batch_start + max_interval, timestamp_end))
            for batch_start in range(timestamp_start, timestamp_end, max_interval)
        ]
        return self._load_energy(
            f"short-energy/{device_id}", device_id, intervals, unit
        )


In [263]:
logger: logging.Logger = get_logger()
public_api_client = PublicApiClient(ENVIRONMENT, API_KEY, MAX_TPS, logger)

# Data downloading

In [ ]:
def is_valid_device_id(device_id: str, raise_error: bool = False) -> tuple[bool, str]:
  """
  Returns True if the device ID is valid, False if not.
  Simplified version based on lib_common
  """
  DEVICE_ID_PATTERN = "^[B-F]{1}[A-F0-9]{12}$"
  DEVICE_ID_REGEX = re.compile(DEVICE_ID_PATTERN)
  # re.match won't detect trailing space in the device id, but re.fullmatch will.
  if not DEVICE_ID_REGEX.fullmatch(device_id):
      return False
  return True

# Determine devices to analyse
devices: list[str] = []
if DEVICE_IDS is not None and len(DEVICE_IDS) > 0:
  devices = DEVICE_IDS
else:
  # get all devices associated with API key
  result, error = public_api_client.get_devices_list()
  if error is not None:
    logger.error(f'failed to load devices for API key: {error}')
  else:
    devices = result

# filter out any invalid device ids
devices =[d for d in devices if is_valid_device_id(d)]

num_devices = len(devices)
logger.info(f'found {num_devices} devices to analyse')

In [265]:
time_start = pendulum.parse(START_DATE, tz=TIMEZONE)
time_end = pendulum.parse(END_DATE, tz=TIMEZONE).add(days=1)

def get_latest_5_min_boundary(dt):
    # Subtract the remainder of minutes to align to the nearest 5-minute boundary
    minutes_to_subtract = dt.minute % 5
    adjusted_time = dt.subtract(minutes=minutes_to_subtract, seconds=dt.second, microseconds=dt.microsecond)
    return adjusted_time

if time_start > time_end:
    raise ValueError(f"Start date ({START_DATE}) has to be earlier than end date ({END_DATE})")
if time_end > pendulum.now(tz=TIMEZONE):
    if time_end > pendulum.now(tz=TIMEZONE).end_of("day").add(seconds=1):
        raise ValueError(f"End date ({END_DATE}) needs to be in the past")
    else:
        # Adjust time_end to now (or previous 5m boundary to make life easier on ourselves)
        time_end = get_latest_5_min_boundary(pendulum.now(tz=TIMEZONE))


In [266]:
# TODO: load device status for each device to exclude decommissioned devices
# Issue: only user-apps-api exposes this and we can't use that because of different auth system

In [267]:
# Transform interval data
def process_interval_data(device_id:str, interval_data:list) -> pd.DataFrame:

  def flatten_arrays(item: dict) -> dict:
      """ flatten each element of arrays to their own key. Other types of values are left untouched.
      e.g. {key: [value0, value1, ...]} becomes {key_0: value0, key_1: value1, ...}.            
      """
      flattened = {}
      for key, value in item.items():
        if isinstance(value, list):
          for idx, subvalue in enumerate(value):
            flattened[f"{key}_{idx}"] = subvalue
        else:
          flattened[key] = value
      return flattened

  intervals = []
  for item in interval_data:
      row = flatten_arrays(item)
      intervals.append(row)

  df_intervals = pd.DataFrame.from_dict(intervals) 
  
  df_intervals["device_id"] = device_id
  # Reorder the columns to move 'device_id', 'timestamp', and 'duration' to the front
  columns_order = ['device_id', 'timestamp', 'duration'] + [col for col in df_intervals.columns if col not in ['device_id', 'timestamp', 'duration']]
  df_intervals = df_intervals[columns_order]

  return df_intervals

In [268]:
# By-day analysis
def roll_up_to_daily(df_intervals: pd.DataFrame) -> pd.DataFrame:
  df_intervals['datetime_end'] = pd.to_datetime(df_intervals['timestamp'], unit='s').dt.tz_localize('UTC').dt.tz_convert(TIMEZONE)
  df_intervals['datetime_start'] = pd.to_datetime(df_intervals['timestamp'] - 300, unit='s').dt.tz_localize('UTC').dt.tz_convert(TIMEZONE)

  # Aggregate based on `datetime_start` to ensure intervals are attributed to the correct day
  df_daily_counts = df_intervals.groupby(['device_id', df_intervals['datetime_start'].dt.date]).size().reset_index(name='entry_count')
  df_daily_counts.columns = ['device_id', 'date', 'num_intervals']
  df_daily_counts['date'] = pd.to_datetime(df_daily_counts['date']).dt.tz_localize(TIMEZONE)

  return df_daily_counts


In [269]:
def add_missing_days(df: pd.DataFrame | None, device_id: str, time_start: DateTime, time_end: DateTime) -> pd.DataFrame:
  interval = pendulum.interval(time_start, time_end.subtract(seconds=1))
  date_range = interval.range('days')
  missing_entries = set([d for d in date_range])
  if df is not None:
    missing_entries = set([d for d in date_range]).difference(set(df['date'].unique()))
  missing_df = pd.DataFrame(list(missing_entries), columns=['date'])
  missing_df['device_id'] = device_id
  missing_df['num_intervals'] = 0
  df = df if missing_df.empty else pd.concat([df, missing_df], ignore_index=True)
  return df


def add_completeness(df: pd.DataFrame, timestamp_first_le: int, timestamp_end: int) -> pd.DataFrame:
  num_intervals_expected_daily = 24 * 12
  df['num_intervals_expected'] = num_intervals_expected_daily

  df['date'] = pd.to_datetime(df['date'], utc=True).dt.tz_convert(TIMEZONE)
  # temporarily add timestamp columns based on date
  df['timestamp_start'] = df['date'].apply(lambda x: int(x.timestamp()))
  df['timestamp_end'] = df['timestamp_start'] + 24 * 3600
  # Adjust timestamp_start for end-of-interval timestamps
  df['timestamp_start'] = df['timestamp_start'] - 300

  # HANDLE PARTIAL DAYS AT START OF INTERVAL

  # when timestamp_end < day of timestamp_first_le, num_intervals_expected = 0
  df.loc[df['timestamp_end'] < timestamp_first_le, 'num_intervals_expected'] = 0

  # when timestamp_first_le is between timestamp_start and timestamp_end, 
  # #num_intervals_expected depends on timestamp_first_le (i.e. num intervals betweem timestamp_first_le and end of day) 
  mask = (df['timestamp_start'] <= timestamp_first_le) & (df['timestamp_end'] > timestamp_first_le)
  # The +1 is there becuause if timestamp_first_le is equal to timestamp_end, that means there is 1 expected interval (the one represented by timestamp_first_le)
  # The same logic holds if timesmtap_fitst_le is smaller than timestamp_end
  df.loc[mask, 'num_intervals_expected'] =  ((df.loc[mask, 'timestamp_end'] - timestamp_first_le) // 300).astype(int) + 1

  # HANDLE PARTIAL DAY AT END OF INTERVAL
  mask = (df['timestamp_end'] > timestamp_end)
  # For partial day at end of interval, follow same logic as for partial days at start of interval 
  # (but subtract the number of intervals between end of day and timestamp_end from the expected number of intervals for a full day)
  df.loc[mask, 'num_intervals_expected'] = df.loc[mask, 'num_intervals_expected'] - (((df.loc[mask, 'timestamp_end'] - timestamp_end) // 300).astype(int) + 1)
  

  df['missing_intervals'] = df['num_intervals_expected'] - df['num_intervals']
  df['interval_completeness'] = df.apply(
      lambda row: 100 if row['num_intervals_expected'] == 0 else 100 * row['num_intervals'] / row['num_intervals_expected'], 
      axis=1
  )
  df['date'] = df['date'].dt.date
  return df

def move_column_to_index(df: pd.DataFrame, column_name: str, index: int) -> pd.DataFrame:
  cols = df.columns.tolist()
  # TODO: add check for column name and num columns
  cols.insert(index, cols.pop(cols.index(column_name)))
  df = df[cols]
  return df

In [ ]:
# Download LE data
def get_le_request_interval(first_le_dict: dict | None, timestamp_start: int, timestamp_end: int) -> tuple | None:
  # If result *is* None, there is no first LE i.e. device not initialised
  # In that case we don't need to query LE and expected num intervals for whole period is 0
  if first_le_dict is None:
    return None
  else:
    first_le_timestamp = first_le_dict['timestamp']
    if first_le_timestamp > timestamp_end:
      # Device not initialised in the requested period
      # No need to query LE and and expected num intervals for whole period is 0
      return None
    elif first_le_timestamp > timestamp_start:
      # Partial data for the requested interval
      # Request from first_le_timestamp, expected num intervals up to first_le_timestamp is 0
      return first_le_timestamp, timestamp_end
    else:
      # Request whole requested interval, expected num intervals is all intervals in the requested interval.
      return timestamp_start, timestamp_end


# Add 5 minutes to time_start and time_end to adjust for fact that interval are *up to* timestamp
timestamp_start = time_start.add(minutes=5).int_timestamp
timestamp_end = time_end.add(minutes=5).int_timestamp

df_unaggregated: pd.DataFrame = pd.DataFrame()
df_daily_counts: pd.DataFrame = pd.DataFrame()
for index, device_id in enumerate(devices):
  logger.info(f'Downloading LE data for device {index+1}/{num_devices} - {device_id}')
  # Get first LE
  result, error = public_api_client.get_first_le(device_id)
  if error is not None:
    logger.error(f'Failed to load first LE for device: {device_id}: {error}')
  else:
    le_request_interval = get_le_request_interval(result, timestamp_start, timestamp_end)
    if le_request_interval is None:
      request_timestamp_start = None
    else:
      request_timestamp_start, request_timestamp_end = le_request_interval
      result, error = public_api_client.load_long_energy(device_id, request_timestamp_start, request_timestamp_end)
      if error is not None:
        logger.error(f'Failed to load LE for device {device_id} between {request_timestamp_start} and {request_timestamp_end}: {error}')
      else:
        if len(result) == 0: 
          logger.info(f"No LE data found for {device_id}")
      
    if result is None or len(result) == 0:   
      df_device_daily = None
    else:
      df_device_intervals = process_interval_data(device_id, result)

      # Process into days, include # intervals, # excpected, # missing
      df_device_daily = roll_up_to_daily(df_device_intervals)

      if ANALYSE_UNAGGREGATED_DATA and len(devices) < 50:
        # Only keep columns of interest

        df_device_intervals = df_device_intervals[["device_id", "timestamp", "duration"]]
        df_unaggregated = df_device_intervals if df_unaggregated.empty else pd.concat([df_unaggregated, df_device_intervals])
      
    df_device_daily = add_missing_days(df_device_daily, device_id, time_start, time_end)
    df_device_daily = add_completeness(df_device_daily, request_timestamp_start, timestamp_end)

    df_daily_counts = df_device_daily if df_daily_counts.empty else pd.concat([df_daily_counts, df_device_daily])
    
logger.info(f'Successfully downloaded LE data for {len(df_daily_counts["device_id"].unique())}/{num_devices} devices')

df_daily_counts = df_daily_counts.reset_index(drop=True)
# TODO: do this on a per device basis, so we can take firstLE into account


# Re-order columns
df_daily_counts = move_column_to_index(df_daily_counts, 'interval_completeness', 2)


In [271]:
# Check whether df_daily is empty
if df_daily_counts.empty:
    raise ValueError("No LE data was downloaded for any device.")

# Analysis


In [272]:

df_aggregated = df_daily_counts.groupby('device_id').agg({
    'num_intervals': 'sum',
    'num_intervals_expected': 'sum'
}).reset_index()

df_daily_counts['date'] = pd.to_datetime(df_daily_counts['date'])
df_daily_counts = df_daily_counts.set_index('date')

# Aggregate by device by month
df_monthly_counts = df_daily_counts.groupby(['device_id', pd.Grouper(freq='ME')]).agg({
    'num_intervals': 'sum',
    'num_intervals_expected': 'sum'
}).reset_index()
df_monthly_counts['date'] = df_monthly_counts['date'].dt.to_period('M').astype(str)

# Aggregate across all devices by day
df_daily_counts_all = df_daily_counts.groupby('date').agg({
    'num_intervals': 'sum',
    'num_intervals_expected': 'sum'
}).reset_index()

# Aggregate across all devices by month
df_daily_counts_all['date'] = pd.to_datetime(df_daily_counts_all['date'])
df_daily_counts_all = df_daily_counts_all.set_index('date')
df_monthly_counts_all = df_daily_counts_all.groupby(pd.Grouper(freq='ME')).agg({
    'num_intervals': 'sum',
    'num_intervals_expected': 'sum'
}).reset_index()
df_monthly_counts_all['date'] = df_monthly_counts_all['date'].dt.to_period('M').astype(str)


def add_completeness_columns(df: pd.DataFrame) -> pd.DataFrame:
    df['num_intervals_missing'] = df['num_intervals_expected'] - df['num_intervals']
    df['interval_completeness'] = 100 * df['num_intervals'] / df['num_intervals_expected']
    # df = df.drop(columns=['num_intervals_expected'])
    if 'date' in df.columns:
        df = df.sort_values(by='date')
    return df

df_aggregated = add_completeness_columns(df_aggregated)

df_monthly_counts = add_completeness_columns(df_monthly_counts)
df_monthly_counts = move_column_to_index(df_monthly_counts, 'interval_completeness', 2)

df_daily_counts_all = add_completeness_columns(df_daily_counts_all)
df_daily_counts_all = df_daily_counts_all.reset_index()
df_daily_counts_all = move_column_to_index(df_daily_counts_all, 'interval_completeness', 1)

df_monthly_counts_all = add_completeness_columns(df_monthly_counts_all)
df_monthly_counts_all = move_column_to_index(df_monthly_counts_all, 'interval_completeness', 1)
  
# We can now remove `num_intervals_expected` from `df_daily_counts` so it's not included in the itable later on
# df_daily_counts = df_daily_counts.drop(columns=['num_intervals_expected'])
df_daily_counts = df_daily_counts.reset_index()


# Devices we couldn't download LE data for (included in devices list but not in LE dict)
devices_with_le_data = list(df_aggregated["device_id"].unique())
devices_without_le_data = [device_id for device_id in devices if device_id not in devices_with_le_data]

# Devices with missing LE data
# TODO: add alternative analysis based on timestamp and duration of interval (only works for intervals between existing intervals, i.e. need to handle missing intervals at start or end of period separately)
# Could also just do a quick analysis to verify all intervals have a duration of 300s.
devices_with_missing_le_data = df_aggregated[df_aggregated['num_intervals_missing'] > 0]['device_id'].tolist()

# Devices not meeting data completeness threshold

devices_not_meeting_threshold = df_aggregated[df_aggregated['interval_completeness'] < DATA_COMPLETENESS_THRESHOLD]['device_id'].tolist()

# Devices with complete LE data
devices_with_complete_le_data = df_aggregated[df_aggregated['num_intervals_missing'] == 0]['device_id'].tolist()


# Determine expected number of intervals over the full analysed period
# Calculate the number of 5-minute intervals between the start and end timestamps using pendulum
num_intervals_expected = int((time_end.diff(time_start).in_minutes()) // 5)

# Devices installed within analysed period
devices_installed_in_period = df_aggregated[(df_aggregated['num_intervals_expected'] > 0) & (df_aggregated['num_intervals_expected'] < num_intervals_expected )]['device_id'].tolist() 

# Devices not installed at all
devices_not_installed = df_aggregated[df_aggregated['num_intervals_expected'] == 0]['device_id'].tolist()



In [273]:
# High level analysis

parameters = [{
  'start_time': time_start,
  'end_time': time_end,
  'num_expected_intervals': num_intervals_expected,
}]

# TODO: update overall completeness calculation as we now take firstLE into account - so different devices have different num_expected_intervals
top_level_stats = [{
  'num_devices': num_devices,
  'overall_completeness': df_aggregated['num_intervals'].sum() / df_aggregated['num_intervals_expected'].sum() * 100,
  'devices_under_threshold': len(devices_not_meeting_threshold),
  'devices_with_missing_intervals': len(devices_with_missing_le_data),
  'devices_without_data': len(devices_without_le_data),
  'devices_with_complete_data': len(devices_with_complete_le_data),
  'devices_installed_during_analysis_period': len(devices_installed_in_period),
  'devices_not_installed': len(devices_not_installed)
}]

df_parameters = pd.DataFrame.from_dict(parameters)
df_stats = pd.DataFrame.from_dict(top_level_stats)

In [274]:
def trend_graph(df: pd.DataFrame) -> plotly.graph_objects.Figure:
  # Calculate the average interval_completeness
  average_completeness = df['interval_completeness'].mean()

  # Create the plot
  fig = px.line(
    df, 
    x='date',
    y='interval_completeness', 
    title='Interval Completeness Over Time'
  )
  # Update the x-axis and y-axis labels
  fig.update_layout(
      xaxis_title='Month',
      yaxis_title='Interval Completeness (%)'
  )

  # Add a horizontal line for the average interval_completeness
  fig.add_shape(
      type="line",
      x0=df['date'].min(),
      y0=average_completeness,
      x1=df['date'].max(),
      y1=average_completeness,
      line=dict(
          color="Red",
          width=2,
          dash="dashdot",
      ),
      name="Average Interval Completeness",
      showlegend=True
  )

  # Add the line graph to the legend
  fig.for_each_trace(
      lambda trace: trace.update(name='Interval Completeness'),
  )
  fig.update_traces(showlegend = True)
  return fig

# Outputs

## Aggregate stats

In [ ]:
show(df_parameters)
show(df_stats, 
     columnDefs= [
        { "targets": [1], "createdCell": JavascriptFunction(
                f"""
                    function (td, cellData, rowData, row, col) {{
                        if (cellData < {DATA_COMPLETENESS_THRESHOLD}) {{
                            $(td).css('color', 'red')
                        }}
                    }}
                """
        )},
        {
            "targets": [1],
            "render": JavascriptCode("$.fn.dataTable.render.number(',', '.', 2, '', '%')"),
        }
    ],)



## Aggregate per month stats

In [ ]:
show(df_monthly_counts_all, 
     columnDefs= [
        { "targets": [1], "createdCell": JavascriptFunction(
                f"""
                    function (td, cellData, rowData, row, col) {{
                        if (cellData < {DATA_COMPLETENESS_THRESHOLD}) {{
                            $(td).css('color', 'red')
                        }}
                    }}
                """
        )},
        {
            "targets": [1],
            "render": JavascriptCode("$.fn.dataTable.render.number(',', '.', 2, '', '%')"),
        }
    ],
    showIndex=False,
    pageLength=20,
    buttons=["copyHtml5", "csvHtml5", "excelHtml5"]
)

In [ ]:
fig = trend_graph(df_monthly_counts_all)
fig.show()

## Aggregate per day stats

In [ ]:
show(df_daily_counts_all, 
     columnDefs= [
        { "targets": [1], "createdCell": JavascriptFunction(
                f"""
                    function (td, cellData, rowData, row, col) {{
                        if (cellData < {DATA_COMPLETENESS_THRESHOLD}) {{
                            $(td).css('color', 'red')
                        }}
                    }}
                """
        )},
        {
            "targets": [1],
            "render": JavascriptCode("$.fn.dataTable.render.number(',', '.', 2, '', '%')"),
        }
    ],
    showIndex=False,
    pageLength=20,
    buttons=["copyHtml5", "csvHtml5", "excelHtml5"]
)

In [ ]:
fig = trend_graph(df_daily_counts_all)
fig.show()

## Per device stats

In [ ]:
df_aggregated = df_aggregated[['device_id', 'interval_completeness', 'num_intervals_missing']]
show(df_aggregated, 
     columnDefs= [
        { "targets": [1], "createdCell": JavascriptFunction(
                f"""
                    function (td, cellData, rowData, row, col) {{
                        if (cellData < {DATA_COMPLETENESS_THRESHOLD}) {{
                            $(td).css('color', 'red')
                        }}
                    }}
                """
        )},
        {
            "targets": [1],
            "render": JavascriptCode("$.fn.dataTable.render.number(',', '.', 2, '', '%')"),
        }
    ],
    showIndex=False,
    buttons=["copyHtml5", "csvHtml5", "excelHtml5"]
)


## Per device per month stats

In [ ]:
show(df_monthly_counts, 
     columnDefs= [
        { "targets": [2], "createdCell": JavascriptFunction(
                f"""
                    function (td, cellData, rowData, row, col) {{
                        if (cellData < {DATA_COMPLETENESS_THRESHOLD}) {{
                            $(td).css('color', 'red')
                        }}
                    }}
                """
        )},
        {
            "targets": [2],
            "render": JavascriptCode("$.fn.dataTable.render.number(',', '.', 2, '', '%')"),
        }
    ],
    showIndex=False,
    pageLength=20,
    buttons=["copyHtml5", "csvHtml5", "excelHtml5"]
)

## Per device per day stats

In [ ]:
show(df_daily_counts, 
     columnDefs= [
        { "targets": [2], "createdCell": JavascriptFunction(
                f"""
                    function (td, cellData, rowData, row, col) {{
                        if (cellData < {DATA_COMPLETENESS_THRESHOLD}) {{
                            $(td).css('color', 'red')
                        }}
                    }}
                """
        )},
        {
            "targets": [2],
            "render": JavascriptCode("$.fn.dataTable.render.number(',', '.', 2, '', '%')"),
        }
    ],
    showIndex=False,
    pageLength=20,
    buttons=["copyHtml5", "csvHtml5", "excelHtml5"]
)

## Analysis of unaggregated data

In [283]:
if df_unaggregated.empty:
    raise ValueError("Unaggregated interval analysis not configured or too many devices selected.")

df_unaggregated = df_unaggregated.reset_index(drop=True)

In [284]:
# Time-of-day across all devices (assumes all devices in same timezone)

In [285]:
def _resample_dataframe(df: pd.DataFrame, resample_interval: str) -> pd.DataFrame:
    df["datetime"] = (
        pd.to_datetime(df["timestamp"], unit="s").dt.tz_localize("UTC").dt.tz_convert(TIMEZONE)
    )
    df = df.set_index("datetime")

    df_resampled = df.resample(resample_interval).agg(
        duration_sum=("duration", "sum"),
        min_timestamp=("timestamp", "min"),
        max_timestamp=("timestamp", "max"),
        num_intervals=("timestamp", "count"),
    )
    return df_resampled

In [286]:
def graph_missing_intervals(df: pd.DataFrame, title: str, x: str="datetime", y: str="num_intervals_missing") -> go.Figure:
    df = df.reset_index()
    fig = px.scatter(
        df,
        x=x,
        y=y,
        labels={"num_intervals_missing": "# missing intervals", "datetime": "Time", "time": "Time"},
        title=title
    )
    missing_intervals_max = df["num_intervals_missing"].max()
    range_max = max(missing_intervals_max * 1.1, 1.1)
    range_min = -0.1 if missing_intervals_max < 5 else -1
    dtick = 1 if missing_intervals_max < 10 else 5
    fig.update_yaxes(range=[range_min, range_max], dtick=dtick)
    return fig

In [ ]:
# Time series per device
df_unaggregated_grouped = df_unaggregated.groupby('device_id')
num_plots: int = len(df_unaggregated_grouped)
fig = make_subplots(
  rows=num_plots, 
  cols=1,
  subplot_titles=[device_id for device_id in df_unaggregated_grouped.groups.keys()]
)
row = 0
for device_id, df_device in df_unaggregated_grouped:
  row += 1
  print(device_id)
  df_device_resampled = _resample_dataframe(df_device, "5min")
  df_device_resampled["num_intervals_expected"] = 1
  df_device_resampled["num_intervals_missing"] = (df_device_resampled["num_intervals_expected"] - df_device_resampled["num_intervals"]).clip(lower=0)
  device_fig = graph_missing_intervals(df_device_resampled, device_id)
  for trace in device_fig.data:
    fig.add_trace(trace, row=row, col=1)
  fig.update_xaxes(title_text = "Time", row=row, col=1)
  fig.update_yaxes(title_text = "# Intervals missing", 
                   row=row, 
                   col=1, 
                   tickvals= [*range(0, max(2, int(df_device_resampled['num_intervals_missing'].max() + 1)))]
                  )
  
fig.update_layout(height=num_plots * 270 + 100)
fig.update_layout(title="Missing Intervals Time Series")

fig.show()


In [ ]:
# Time-of-day per device

df_unaggregated_grouped = df_unaggregated.groupby('device_id')
num_plots: int = len(df_unaggregated_grouped)
fig = make_subplots(
  rows=num_plots, 
  cols=1,
  subplot_titles=[f"{device_id} ({START_DATE} - {END_DATE})" for device_id in df_unaggregated_grouped.groups.keys()],
)
row = 0
for device_id, df_device in df_unaggregated_grouped:
  row += 1
  print(device_id)
  df_device_resampled = _resample_dataframe(df_device, "5min")
  df_device_resampled["num_intervals_expected"] = 1
  df_device_resampled["num_intervals_missing"] = (df_device_resampled["num_intervals_expected"] - df_device_resampled["num_intervals"]).clip(lower=0)
  # Add time-of-day column
  df_device_resampled['time'] = df_device_resampled.index.time
  # Group by time of day and aggregate num_intervals_missing
  df_time_of_day_grouped = df_device_resampled.groupby('time').agg(num_intervals_missing=('num_intervals_missing', 'sum')).reset_index()
  
  device_fig = graph_missing_intervals(df_time_of_day_grouped, f"{device_id} ({START_DATE} - {END_DATE})", "time")
  for trace in device_fig.data:
    fig.add_trace(trace, row=row, col=1)
  fig.update_xaxes(title_text = "Time of day", row=row, col=1)
  fig.update_yaxes(title_text = "# Intervals missing", 
                   row=row, col=1, 
                   tickvals= [*range(0, max(2, int(df_time_of_day_grouped['num_intervals_missing'].max() + 1)))]
                  )

fig.update_layout(height=num_plots * 270 + 100)
fig.update_layout(title="Missing Intervals by Time of Day")


fig.show()